## 전국 신규 민간아파트 분양가격 동향

* youtube: https://www.youtube.com/watch?v=H6-Y-sipgCk&list=PLaTc2c6yEwmpoDDdE9C1p0xfQXz6eEzmH
* data: https://www.data.go.kr/dataset/3035522/fileData.do

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import re
from plotnine import *

In [ ]:
pre_sale = pd.read_csv('data/bunyangga.csv', encoding='euc-kr')
pre_sale.shape

In [ ]:
pre_sale.info()

In [ ]:
pre_sale.isnull().sum()

In [ ]:
pre_sale['연도'] = pre_sale['연도'].astype(str)
pre_sale['월'] = pre_sale['월'].astype(str)

In [ ]:
pre_sale_price = pre_sale['분양가격(㎡)']
pre_sale['분양가격'] = pd.to_numeric(pre_sale_price, errors='coerse')

In [ ]:
pre_sale['평당분양가격'] = pre_sale['분양가격'] * 3.3

In [ ]:
pre_sale.info()

In [ ]:
pre_sale.isnull().sum()

In [ ]:
pre_sale.describe(include=[np.object])

In [ ]:
pre_sale_2017 = pre_sale[pre_sale['연도']=='2017']
pre_sale_2017.shape

In [ ]:
pre_sale['규모구분'].value_counts()

### 전국 평균 분양 가격

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
pre_sale.groupby(pre_sale.연도).describe().T

In [ ]:
pre_sale.pivot_table('평당분양가격', '규모구분', '연도')

### 전국 분양가 변동 금액

In [ ]:
region_year_all = pre_sale[pre_sale['규모구분'] == '전체']
region_year_all.shape

In [ ]:
region_year = region_year_all.pivot_table('평당분양가격', '지역명', '연도').reset_index()
region_year.head()

In [ ]:
region_year['변동액'] = (region_year['2018'] - region_year['2015']) * 1000

In [ ]:
region_min = region_year.iloc[region_year['변동액'].argmin()]
region_max = region_year.iloc[region_year['변동액'].argmax()]

print(region_min['지역명'], region_min['변동액'])
print(region_max['지역명'], region_max['변동액'])


### 시각화

In [ ]:
(ggplot(region_year_all)
 + aes(x='지역명', y='평당분양가격', fill='연도')
 + geom_bar(stat='identity', position='dodge')
 + ggtitle('2015-2018 신규 민간아파트 분양가격')
 + theme(text=element_text(family='NanumBarunGothic'), figure_size=(8, 4))
)

In [ ]:
pre_sale.pivot_table('평당분양가격', '규모구분', '지역명')

In [ ]:
# '평당분양가격', '규모구분', '지역명'
(ggplot(pre_sale)
 + aes(x='지역명', y='평당분양가격', fill='규모구분')
 + geom_bar(stat='identity', position='dodge')
 + ggtitle('규모별 신규민간 아파트 분양가격')
 + theme(text=element_text(family='NanumBarunGothic'), figure_size=(8,4))
)

In [ ]:
(ggplot(pre_sale)
 + aes(x='연도', y='평당분양가격', fill='규모구분')
 + geom_bar(stat='identity', position='dodge')
 + facet_wrap('지역명')
 + theme(text=element_text(family='NanumBarunGothic'),
         axis_text_x=element_text(rotation=70), figure_size=(12,12))
)

In [ ]:
(ggplot(pre_sale)
 + aes(x='지역명', y='평당분양가격', fill='규모구분')
 + geom_boxplot()
 + ggtitle('2015-2018 신규 민간아파트 분양가격')
 + theme(text=element_text(family='NanumBarunGothic'), figure_size=(12, 6))
)

In [ ]:
pre_sale_seoul = pre_sale.loc[pre_sale['지역명']=='서울']
(ggplot(pre_sale_seoul)
 + aes(x='연도', y='평당분양가격', fill='규모구분')
 + ggtitle('서울')
 + geom_boxplot()
 + theme(text=element_text(family='NanumBarunGothic'))
)

In [ ]:
pre_sale_seoul.head()

#### tidy data

In [ ]:
df = pd.read_csv('data/bunyangga_pre.csv', encoding='euc-kr', skiprows=1, header=0)
df.shape

In [ ]:
df

In [ ]:
year = df.iloc[0]; month = df.iloc[1]
print(year); print('------'); print(month)

In [ ]:
for i, v in enumerate(year):
    if i > 2 and i < 15:
        year[i] = ' '.join(['2014년', month[i]])
    elif i >= 15:
        year[i] = ' '.join(['2015년', str(month[i])])
    elif i == 2:
        year[i] = ' '.join([year[i], month[i]])
    elif i == 1:
        year[i] = '시군구'

In [ ]:
year

In [ ]:
df.columns = year

In [ ]:
df = df.drop(df.index[[0, 1]])

In [ ]:
df['구분'] = df['구분'].fillna('')
df['시군구'] = df['시군구'].fillna('')

In [ ]:
df['지역'] = df['구분'] + df['시군구']

In [ ]:
df['지역']

In [ ]:
melt_columns = df.columns.copy()
melt_columns

In [ ]:
df_2013_2015 = pd.melt(df, id_vars=['지역'], value_vars=['2013년 12월', '2014년 1월', '2014년 2월', '2014년 3월',
       '2014년 4월', '2014년 5월', '2014년 6월', '2014년 7월', '2014년 8월', '2014년 9월',
       '2014년 10월', '2014년 11월', '2014년 12월', '2015년 1월', '2015년 2월',
       '2015년 3월', '2015년 4월'])

In [ ]:
df_2013_2015.head(5)

In [ ]:
df_2013_2015.columns = ['지역','기간','분양가']

In [ ]:
df_2013_2015.head(5)

In [ ]:
df_2013_2015['연도'] = df_2013_2015['기간'].apply(lambda x: x.split('년')[0])
df_2013_2015['월'] = df_2013_2015['기간'].apply(lambda x: x.split('년')[1].strip())

In [ ]:
df_2013_2015['지역'].value_counts()

In [ ]:
import re
df_2013_2015['지역'] = df_2013_2015['지역'].apply(lambda x: re.sub('6대광역시부산', '부산', x))
df_2013_2015['지역'] = df_2013_2015['지역'].apply(lambda x: re.sub('지방강원', '강원', x))

In [ ]:
df_2013_2015['지역'].value_counts()

In [ ]:
df_2013_2015.info()

In [ ]:
df_2013_2015['분양가격'] = df_2013_2015['분양가'].str.replace(',', '').astype(int)

In [ ]:
(ggplot(df_2013_2015)
 + aes(x='지역', y='분양가격', fill='연도')
 + geom_boxplot()
 + theme(text=element_text(family='NanumBarunGothic'), figure_size=(12, 4))
)